In [3]:
from datasets import load_dataset
dataset = load_dataset('DBD-research-group/BirdSet','HSN', download_mode='force_redownload')

ModuleNotFoundError: No module named 'datasets'

In [52]:
from birdset.datamodule.base_datamodule import DatasetConfig
from birdset.datamodule.birdset_datamodule import BirdSetDataModule

# initiate the data module
dm = BirdSetDataModule(
    dataset= DatasetConfig(
        data_dir='/workspace/data_birdset/HSN',
        dataset_name='HSN',
        hf_path='DBD-research-group/BirdSet',
        hf_name='HSN',
        n_classes=21,
        n_workers=3,
        val_split=0.2,
        task="multilabel",
        classlimit=500,
        eventlimit=5,
        sampling_rate=32000,
    ),
)
# prepare the data (download dataset, ...)
dm.prepare_data()
# setup the dataloaders
dm.setup(stage="fit")
# get the dataloaders
train_loader = dm.train_dataloader()
# get the first batch
batch = next(iter(train_loader))
# get shape of the batch
print(batch["input_values"].shape)
print(batch["labels"].shape)
batch
   

Processing labels: 100%|██████████| 21/21 [00:02<00:00, 10.29it/s]


Map (num_proc=3):   0%|          | 0/17940 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14352 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3588 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12000 [00:00<?, ? examples/s]

torch.Size([32, 1, 128, 1024])
torch.Size([32, 21])


{'input_values': tensor([[[[ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           ...,
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752]]],
 
 
         [[[ 4.1408,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 2.9707,  5.1379,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 3.3263,  5.6399,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           ...,
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752],
           [ 6.2264,  6.2264,  6.2264,  ..., 16.5752, 16.5752, 16.5752]]],
 
 
         [[[-3.0554, -1.1465,  0.1514,  ..., 16.5752, 16.5752, 1

In [53]:
import torch
import torch.nn as nn
import torchvision
from typing import List, Literal, Tuple

In [54]:
from lightning import Trainer ,LightningModule

min_epochs = 1
max_epochs = 5
trainer = Trainer(min_epochs=min_epochs, max_epochs=max_epochs, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [55]:
print(dm.len_trainset,
    dm.task,
    dm.train_batch_size,
    max_epochs)

14352 multilabel 32 5


In [56]:

from birdset.modules.models.resnet import ResNetClassifier
from birdset.modules.metrics.multilabel import MultilabelMetricsConfig
from torch.nn import BCEWithLogitsLoss
from birdset.modules.multilabel_module import MultilabelModule
module = ResNetClassifier("resnet50",21)
#import torch
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True),

from birdset.modules.base_module import BaseModule,NetworkConfig
NetworkConfig=NetworkConfig(
        model=ResNetClassifier(baseline_architecture="resnet50",num_classes =21),
        model_name ="resnet50",
        model_type="vision",
        torch_compile= False,
        sample_rate=32000,
        normalize_waveform=False,
        normalize_spectrogram=True)


model = BaseModule(
    network=NetworkConfig,
    loss=BCEWithLogitsLoss(),
    metrics=MultilabelMetricsConfig(),
    len_trainset=dm.len_trainset,
    task=dm.task,
    batch_size=dm.train_batch_size,
    num_epochs=max_epochs)



In [57]:
model

BaseModule(
  (loss): BCEWithLogitsLoss()
  (model): ResNetClassifier(
    (model): ResNet(
      (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [58]:
trainer.fit(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name                  | Type              | Params
------------------------------------------------------------
0 | loss                  | BCEWithLogitsLoss | 0     
1 | model                 | ResNetClassifier  | 23.5 M
2 | train_metric          | cmAP              | 0     
3 | valid_metric          | cmAP              | 0     
4 | test_metric           | cmAP              | 0     
5 | valid_metric_best     | MaxMetric         | 0     
6 | valid_add_metrics     | MetricCollection  | 0     
7 | test_add_metrics      | MetricCollection  | 0     
8 | test_complete_metrics | MetricCollection  | 0     
------------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.179    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
